<a href="https://colab.research.google.com/github/waltatgit/CTCModel/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd '/content/drive/My Drive/Colab Notebooks/CTCModel'

/content/drive/My Drive/Colab Notebooks/CTCModel


In [6]:
!tar xvf ./seqDigits.pkl.tar.gz

seqDigits.pkl


In [7]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/CTCModel')

In [8]:
from keras.layers import TimeDistributed, Activation, Dense, Input, Bidirectional, LSTM, Masking, GaussianNoise
from keras.optimizers import Adam
from CTCModel import CTCModel
import pickle
from keras.preprocessing import sequence
import numpy as np

Using TensorFlow backend.


In [9]:
def create_network(nb_features, nb_labels, padding_value):

    # Define the network architecture
    input_data = Input(name='input', shape=(None, nb_features)) # nb_features = image height

    masking = Masking(mask_value=padding_value)(input_data)
    noise = GaussianNoise(0.01)(masking)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(noise)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)

    dense = TimeDistributed(Dense(nb_labels + 1, name="dense"))(blstm)
    outrnn = Activation('softmax', name='softmax')(dense)

    network = CTCModel([input_data], [outrnn])
    network.compile(Adam(lr=0.0001))

    return network

In [10]:
    (x_train, y_train), (x_test, y_test) = pickle.load(open('./seqDigits.pkl', 'rb'))


In [11]:
    nb_labels = 10 # number of labels (10, this is digits)
    batch_size = 32 # size of the batch that are considered
    padding_value = 255 # value for padding input observations
    nb_epochs = 10 # number of training epochs
    nb_train = len(x_train)
    nb_test = len(x_test)
    nb_features = len(x_train[0][0])

In [12]:
    # create list of input lengths
    x_train_len = np.asarray([len(x_train[i]) for i in range(nb_train)])
    x_test_len = np.asarray([len(x_test[i]) for i in range(nb_test)])
    y_train_len = np.asarray([len(y_train[i]) for i in range(nb_train)])
    y_test_len = np.asarray([len(y_test[i]) for i in range(nb_test)])

    # pad inputs
    x_train_pad = sequence.pad_sequences(x_train, value=float(padding_value), dtype='float32',
                                         padding="post", truncating='post')
    x_test_pad = sequence.pad_sequences(x_test, value=float(padding_value), dtype='float32',
                                        padding="post", truncating='post')
    y_train_pad = sequence.pad_sequences(y_train, value=float(nb_labels),
                                         dtype='float32', padding="post")
    y_test_pad = sequence.pad_sequences(y_test, value=float(nb_labels),
                                        dtype='float32', padding="post")

In [13]:
    # define a recurrent network using CTCModel
    network = create_network(nb_features, nb_labels, padding_value)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.cast` instead.


In [ ]:
 # CTC training
network.fit(x=[x_train_pad, y_train_pad, x_train_len, y_train_len], y=np.zeros(nb_train), \
                batch_size=batch_size, epochs=nb_epochs)



Epoch 1/10
 7328/10000 [====================>.........] - ETA: 1:37 - loss: 18.9409